In [ ]:
# Read data from pdf

from pypdf import PdfReader

def extract_text(path):
  reader = PdfReader(path)
  text = ''

  for page in reader.pages:
    text += page.extract_text() + '\n'
  return text

In [ ]:
text = extract_text('The Finance Bill 2025.pdf')
text

In [ ]:
def chunk_text(text, size=800):
  words = text.split()
  chunks=[]
  for i in range(0, len(words), size):
    chunks.append(" ".join(words[i:i+size]))
  return chunks

chunks = chunk_text(text)

In [ ]:
chunks

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('BAAI/bge-small-en-v1.5')

In [ ]:
finance_embeddings = model.encode(chunks).tolist()

finance_embeddings

In [ ]:
len(finance_embeddings)

In [ ]:
import chromadb

client = chromadb.PersistentClient(path='./chromadb')

In [ ]:
collection = client.create_collection(
    name='kenyan_finance_bills',
    metadata={'description':'Kenyan Finance Bills for the "Wantam" era'}
)

print('The collection has been created:', collection.name)

In [ ]:
ids = [f"doc_{i}" for i in range(len(chunks))]

ids

In [ ]:
len(chunks) == len(ids)

In [ ]:
collection.add(
    documents=chunks,
    embeddings=finance_embeddings,
    ids=ids
)

In [ ]:
collection.count()

In [ ]:
query = 'What does the Finance bill 2025 say about taxes'

In [ ]:
query_embedding = model.encode(query).tolist()
query_embedding

In [ ]:
results = collection.query(
    query_embeddings=query_embedding,
    n_results=3
)

results